# 2. Score classic methods

In [5]:
import ast
import pandas as pd

from common import score_models, clean_string, load_preds

In [6]:
reading_lists = pd.read_csv("../reading_lists.csv")
reading_lists['reading_list'] = reading_lists['reading_list'].apply(ast.literal_eval)

trues = { id_:[clean_string(ref["title"]) for ref in references if ref["acl_id"]] for id_, references in zip(reading_lists["id"], reading_lists["reading_list"]) }

In [11]:
models_paths = {
    "Semantic Scholar (Any)": "classic_methods/preds/semantic_scholar/any",
    "Semantic Scholar (Most Cited)": "classic_methods/preds/semantic_scholar/most_cited",
    "Semantic Scholar (ACL)": "classic_methods/preds/semantic_scholar/acl",
    "BM25": "classic_methods/preds/bm25",
    "SPECTER2": "classic_methods/preds/specter2",
}

## Default comparison

In [12]:
table = score_models(trues, models_paths.keys(), models_paths.values(), split_by_years=False, use_title_instead_of_id=True); table 
#print(table.to_latex(float_format="{:0.1f}".format))

recall                                 \
                                     A1        A2         A3      Mean   
Semantic Scholar (Any)         3.313764  2.720157   4.099804  3.377908   
Semantic Scholar (Most Cited)  2.491846  3.356164   5.639269  3.829093   
Semantic Scholar (ACL)         6.526419  4.951076   7.354860  6.277452   
BM25                           8.873668  8.489889  11.156773  9.506777   
SPECTER2                       5.787671  5.676778   8.245864  6.570104   

                                   ndcg                                \
                                     A1        A2        A3      Mean   
Semantic Scholar (Any)         1.472563  1.537772  2.069279  1.693204   
Semantic Scholar (Most Cited)  1.834147  2.907163  4.027345  2.922885   
Semantic Scholar (ACL)         4.199856  3.669359  4.991881  4.287032   
BM25                           5.722704  5.105861  6.581614  5.803393   
SPECTER2                       3.064759  3.397064  4.709815  3.723880   

                                    mrr                                
                                     A1        A2        A3      Mean  
Semantic Scholar (Any)         1.342316  2.037671  2.452423  1.944137  
Semantic Scholar (Most Cited)  3.302892  3.847703  5.165253  4.105283  
Semantic Scholar (ACL)         5.451373  5.287019  6.995543  5.911312  
BM25                           8.765831  5.929288  9.199094  7.964738  
SPECTER2                       3.333865  4.578341  5.763281  4.558496

## Matches of correct prediction

In [13]:
def match(trues:dict, preds1:dict, preds2:dict) -> float:
    match = 0
    size = 0
    for k, trues_v in trues.items():
        if trues_v != []:
            match1 = set(dict.fromkeys(trues_v)) & set(dict.fromkeys(preds1[k]))
            match2 = set(dict.fromkeys(trues_v)) & set(dict.fromkeys(preds2[k]))
            match += len(match1 & match2)
            size += 1
    return match/size

In [14]:
bm25_preds = load_preds("classic_methods/preds/bm25", use_title_instead_of_id=True)
specter2_preds = load_preds("classic_methods/preds/specter2", use_title_instead_of_id=True)

(match(trues, bm25_preds["A1"], specter2_preds["A1"]) +\
match(trues, bm25_preds["A2"], specter2_preds["A2"]) +\
match(trues, bm25_preds["A3"], specter2_preds["A3"])) /3

0.1050228310502283